In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist=input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
batch_size=128
n_batch=mnist.train.num_examples // batch_size

def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1], strides=[1,2,2,1],padding='SAME')



with tf.name_scope('input'):
##定义placeholder
    x=tf.placeholder(tf.float32,[None,784], name='x-input')
    y=tf.placeholder(tf.float32,[None,10], name='y-input')
    keep_prob=tf.placeholder(tf.float32, name='dropout')
    x_image=tf.reshape(x,[-1,28,28,1])
    
with tf.name_scope('layer'):
    w_cov1=weight_variable([5,5,1,32])
    b_cov1=bias_variable([32])
    
    h_cov1=tf.nn.relu(conv2d(x_image,w_cov1)+b_cov1)
    h_pool1=max_pool_2x2(h_cov1)
    
    w_cov2=weight_variable([5,5,32,64])
    b_cov2=bias_variable([64])
    
    h_cov2=tf.nn.relu(conv2d(h_pool1,w_cov2)+b_cov2)
    h_pool2=max_pool_2x2(h_cov2)
    
    w_fc1=weight_variable([7*7*64,1024])
    b_fc1=bias_variable([1024])
    
    h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
    h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)
    
    w_fc2=weight_variable([1024,10])
    b_fc2=bias_variable([10])
    prediction=tf.nn.softmax(tf.matmul(h_fc1_drop, w_fc2)+b_fc2)

with tf.name_scope('loss'):
##二次代价函数
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
    tf.summary.scalar('loss', loss)
    
with tf.name_scope('train_step'):
    train_step=tf.train.AdamOptimizer(1e-3).minimize(loss)

init = tf.global_variables_initializer()

with tf.name_scope('correct_prediction'):
    correct_prediction=tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))

with tf.name_scope('accuarcy'):
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('accuracy', accuracy)

merged=tf.summary.merge_all()
    
with tf.Session() as sess:
    sess.run(init)
    writer=tf.summary.FileWriter('logs/',sess.graph)
    for epoch in range(21):
        for batch in range(n_batch):
            batch_xs, batch_ys= mnist.train.next_batch(batch_size)
            summary, _ = sess.run([merged,train_step], feed_dict={x:batch_xs, y:batch_ys, keep_prob:0.5})
            
        writer.add_summary(summary, epoch)    
        acc=sess.run(accuracy, feed_dict={x:mnist.test.images, y:mnist.test.labels,keep_prob:1})
        print("iter" + str(epoch)+", testing accuracy:" + str(acc))
    

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

iter0, testing accuracy:0.9755
iter1, testing accuracy:0.9789
iter2, testing accuracy:0.9867
iter3, testing accuracy:0.9832
iter4, testing accuracy:0.9867
iter5, testing accuracy:0.9871
iter6, testing accuracy:0.9879
iter7, testing accuracy:0.9901
iter8, testing accuracy:0.9896
iter9, testing accuracy:0.989
iter10, testing accuracy:0.9896
iter11, testing accuracy:0.991
iter12, testing accuracy:0.9896
iter13, testing accuracy:0.9904
iter14, testing accuracy:0.991
iter15, testing accuracy:0.9903
iter16, testing accuracy:0.9919
iter17, testing accuracy:0.9898
iter18, testing accuracy:0.9876
iter19, testing accuracy:0.9925
iter20, testing accuracy:0.9911
